In [ ]:
# Bruno's Sentiment Analysis With Neural Networks

In [7]:
pip install Keras-Preprocessing

ERROR: Could not find a version that satisfies the requirement Keras-Preprocessing (from versions: none)
ERROR: No matching distribution found for Keras-Preprocessing
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

In [9]:
threads_reviews = pd.read_csv('/kaggle/input/threads-app-reviews-dataset/threads_reviews.csv')
threads_reviews.head()

,source,review_description,rating,review_date
0,Google Play,Very good app for Android phone and me,5,27-08-2023 10:31
1,Google Play,Sl👍👍👍👍,5,27-08-2023 10:28
2,Google Play,Best app,5,27-08-2023 9:47
3,Google Play,Gatiya app,1,27-08-2023 9:13
4,Google Play,Lit bruv,5,27-08-2023 9:00


### Categorize Sentiment by Rating

In [10]:
def categorize_sentiment(rating):
    if rating >= 4:
        return 'positive'
    elif rating == 3:
        return 'neutral'
    else:
        return 'negative'

threads_reviews['sentiment'] = threads_reviews['rating'].apply(categorize_sentiment)

threads_reviews.head()

,source,review_description,rating,review_date,sentiment
0,Google Play,Very good app for Android phone and me,5,27-08-2023 10:31,positive
1,Google Play,Sl👍👍👍👍,5,27-08-2023 10:28,positive
2,Google Play,Best app,5,27-08-2023 9:47,positive
3,Google Play,Gatiya app,1,27-08-2023 9:13,negative
4,Google Play,Lit bruv,5,27-08-2023 9:00,positive


### Preparing Data for Model Training

In [11]:
X = threads_reviews['review_description']
y = threads_reviews['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

X_train.shape, X_test.shape

((32348,), (8087,))

#### Tokenizing Text

In [12]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_sequences, maxlen=max_len, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_len, padding='post', truncating='post')

X_train_padded.shape, X_test_padded.shape

ModuleNotFoundError: No module named 'keras_preprocessing'

#### Neural Network Architecture

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, GlobalAveragePooling1D, Dense

model = Sequential([
    Embedding(max_words, 24, input_length=max_len),
    GlobalAveragePooling1D(),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)

#### Run model 15 times and check its accuracy

In [ ]:
history = model.fit(X_train_padded, y_train_categorical, epochs=15, validation_data=(X_test_padded, y_test_categorical), batch_size=32)

In [ ]:
loss, accuracy = model.evaluate(X_test_padded, y_test_categorical)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [ ]:
model.save('sentiment_analysis_threads.keras')